In [1]:
import gensim 

In [2]:
import pandas as pd

In [3]:
wiki_data = pd.read_csv('data/wikipedia/cleaned_wiki_data_full_text_chunks.csv')

In [4]:
def read_corpus(list_of_docs, tokens_only=False):
    for i, line in enumerate(list_of_docs):
        if tokens_only:
            yield gensim.utils.simple_preprocess(line)
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [49]:
wiki_data

,Unnamed: 0,wiki_title,text
0,0,April--0,April is the th month of the year and comes b...
1,1,April--1,April comes between March and May making it th...
2,2,August--0,August is the th month of the year in the Gre...
3,3,August--1,This month was first called Sextilis in Latin ...
4,4,Art--0,Art is a creative activity that expresses ima...
5,5,Art--1,Art is divided into the plastic arts where som...
6,6,Art--2,Some people say that art is a product or item ...
7,7,Art--3,There are sculptures cave painting and rock ar...
8,8,Art--4,In some society societies people think that ar...
9,9,Art--5,The wikt simple function functions of art incl...


In [51]:
removed_duplicates = wiki_data.text.drop_duplicates()

In [52]:
len(removed_duplicates)

152497

In [53]:
train_corpus = list(read_corpus(removed_duplicates))
test_corpus = list(read_corpus(removed_duplicates, tokens_only=True))

In [56]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=10, epochs=40)

In [57]:
model.build_vocab(train_corpus)

In [58]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 1h 38min 16s, sys: 1min 51s, total: 1h 40min 7s
Wall time: 18h 2min 9s


In [59]:
statement = gensim.utils.simple_preprocess('The domestic cat (Felis silvestris catus or Felis catus)[1][4] is a small, typically furry, carnivorous mammal. They are often called house cats[5] when kept as indoor pets or simply cats when there is no need to distinguish them from other felids and felines. They are often valued by humans for companionship and for their ability to hunt vermin. There are more than seventy cat breeds recognized by various cat registries.')
# statement = gensim.utils.simple_preprocess('Bouldering is a form of rock climbing that is performed on small rock formations or artificial rock walls, known as boulders, without the use of ropes or harnesses. While it can be done without any equipment, most climbers use climbing shoes to help secure footholds, chalk to keep their hands dry and provide a firmer grip, and bouldering mats to prevent injuries from falls. Unlike free solo climbing, which is also performed without ropes, bouldering problems (the sequence of moves that a climber performs to complete the climb) are usually less than 6 meters (20 ft.) tall. Traverses, which are a form of boulder problem, require the climber to climb horizontally from one end to another. [1]:3 Artificial climbing walls allow boulderers to train indoors in areas without natural boulders. In addition, Bouldering competitions take place in both indoor and outdoor settings')
# statement = gensim.utils.simple_preprocess('Wrestling is a combat sport involving grappling type techniques such as clinch fighting, throws and takedowns, joint locks, pins and other grappling holds. The sport can either be theatrical for entertainment (see professional wrestling), or genuinely competitive. A wrestling bout is a physical competition, between two (occasionally more) competitors or sparring partners, who attempt to gain and maintain a superior position. There are a wide range of styles with varying rules with both traditional historic and modern styles. Wrestling techniques have been incorporated into other martial arts as well as military hand-to-hand combat systems.')
test_vector = model.infer_vector(statement)

In [60]:
model.docvecs.most_similar([test_vector])

/home/wmvoje/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[(1575, 0.391191691160202),
 (1576, 0.37748217582702637),
 (1583, 0.375293493270874),
 (120619, 0.3728700578212738),
 (8789, 0.364887535572052),
 (37487, 0.36357277631759644),
 (6798, 0.3573053479194641),
 (59597, 0.3570224642753601),
 (75366, 0.35679906606674194),
 (17644, 0.3517945408821106)]

In [63]:
test = clean(test_corpus[1583])
pos_tag(test)

[('house', 'NN'),
 ('cats', 'NNS'),
 ('eat', 'VBP'),
 ('food', 'NN'),
 ('owners', 'NNS'),
 ('give', 'VBP'),
 ('food', 'NN'),
 ('manufacture', 'NN'),
 ('designed', 'VBN'),
 ('nutrient', 'JJ'),
 ('cats', 'NNS'),
 ('types', 'VBP'),
 ('cat', 'VBN'),
 ('food', 'NN'),
 ('come', 'NN'),
 ('flavors', 'NNS'),
 ('costs', 'NNS'),
 ('moist', 'VBP'),
 ('canned', 'VBN'),
 ('food', 'NN'),
 ('cat', 'NN'),
 ('food', 'NN'),
 ('comes', 'VBZ'),
 ('cans', 'NNS'),
 ('bags', 'JJ'),
 ('formulas', 'NNS'),
 ('kitten', 'VBP'),
 ('formulas', 'JJ'),
 ('cat', 'NN'),
 ('health', 'NN'),
 ('formulas', 'NNS'),
 ('formulas', 'VBP'),
 ('reducing', 'VBG'),
 ('cat', 'NN'),
 ('weight', 'VBD'),
 ('others', 'NNS'),
 ('vegetables', 'NNS'),
 ('salmon', 'RB'),
 ('meat', 'VBP'),
 ('milk', 'NN'),
 ('essence', 'NN'),
 ('food', 'NN'),
 ('meat', 'NN'),
 ('cat', 'NN'),
 ('diet', 'JJ'),
 ('make', 'VBP'),
 ('cat', 'NN'),
 ('fed', 'VBN'),
 ('dog', 'JJ'),
 ('food', 'NN'),
 ('make', 'VBP'),
 ('cat', 'NN'),
 ('eyes', 'NNS')]

In [44]:
test_corpus[46589]

['bütgenbach',
 'or',
 'butgenbach',
 'is',
 'municipality',
 'in',
 'the',
 'belgium',
 'belgian',
 'province',
 'of',
 'liège',
 'liège',
 'in',
 'people',
 'lived',
 'there',
 'werkelijke',
 'bevolking',
 'per',
 'gemeente',
 'op',
 'januari',
 'population',
 'de',
 'droit',
 'par',
 'commune',
 'au',
 'janvier',
 'it',
 'is',
 'at',
 'north',
 'east',
 'geografische',
 'coördinaten',
 'van',
 'de',
 'gemeenten']

In [43]:
test_corpus[46586]

['büllingen',
 'or',
 'bullange',
 'is',
 'municipality',
 'in',
 'the',
 'belgium',
 'belgian',
 'province',
 'of',
 'liège',
 'liège',
 'in',
 'people',
 'lived',
 'there',
 'werkelijke',
 'bevolking',
 'per',
 'gemeente',
 'op',
 'januari',
 'population',
 'de',
 'droit',
 'par',
 'commune',
 'au',
 'janvier',
 'it',
 'is',
 'at',
 'north',
 'east',
 'geografische',
 'coördinaten',
 'van',
 'de',
 'gemeenten']

In [64]:
import pickle as p

with open('models/simple_wiki_chunked_corpus_10_count_cutoff.p', 'wb') as topick:
    p.dump(test_corpus, topick)


In [12]:
import os
import random
import codecs
from collections import defaultdict

from gensim.models.ldamodel import LdaModel as Lda
from gensim import corpora
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk import pos_tag

import enchant
spelling_dict = enchant.Dict("en_US")

stop = set(stopwords.words('english'))
lemma = WordNetLemmatizer()
stemmer = PorterStemmer()

stemmed_dict = defaultdict(set)

def stem_and_update_stem_dict(tokens):
    output_list = []
    for token in tokens:
        stemmed = stemmer.stem(token)
        if stemmed != token:
            stemmed_dict[stemmed].add(token)
        output_list.append(stemmed)
    return output_list
        
list_of_POS_to_ignore = ['WRB', 'WP$', 'WP',  'WDT', 'UH', 
                         'TO', 'RP', 'RBS', 'RB', 'RBR', 'PRP$', 'PRP', 
                        'MD', 'JJS', 'JJR', 'JJ', 'IN', 'FW', 'EX', 
                         'DT', 'CD']

# Function to remove stop words from sentences & lemmatize verbs. 
def clean(doc):
    #removing stop words 
    tokens = [i for i in doc if i not in stop]
    
    # removing pos data 
    tokens = [word for word, pos in pos_tag(tokens) if pos not in list_of_POS_to_ignore]
    # Removing improperly spelled words (pronouns must be capitalized to be spelled right)
    tokens = [word for word in tokens if spelling_dict.check(word)]
    # lowercase
    tokens = [word.lower() for word in tokens]
    # lemmatized
#     tokens = [lemma.lemmatize(word, 'v') for word in tokens]
    # removing short words 
    tokens = [s for s in tokens if len(s) > 2]
    # stemmed
#     tokens = stem_and_update_stem_dict(tokens)
    
    return tokens

In [67]:
model.save('models/simple_wiki_chunked_doc2vec_200_vector_10_min_word')